In [30]:
import tensorflow as tf
import os
import tfx
from tfx.components import ImportExampleGen , CsvExampleGen
from tfx.orchestration import metadata
from tfx.proto import example_gen_pb2, trainer_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip



In [31]:
base_dir = os.getcwd()
Tf_record_data_dir = os.path.join(os.pardir, "tfrecord_data")

In [32]:
tf_record_location = os.path.join(base_dir, Tf_record_data_dir)
tf_record_location

'C:\\Users\\ASUS\\building-machine-learning-pipelines\\Untitled Folder\\..\\tfrecord_data'

In [33]:
example_gen = ImportExampleGen(input_base=tf_record_location)#ImportExampleGen because we are impoting Tfrecord file format

In [34]:
context = InteractiveContext(pipeline_root='../pipeline_root_learning/tfx_for_tfrecordData')   # ".." means the parent directory
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [35]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [36]:
from tfx.components import SchemaGen 
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Transform

In [37]:
# Set the transform module filename
_housing_transform_module_file = 'housing_transform.py'

In [38]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'], # tfExample
    schema=schema_gen.outputs['schema'],# Schema
    module_file=os.path.abspath(_housing_transform_module_file)) # preprocessing_fn()



In [40]:
# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Tuner

In [41]:
_tuner_module_file = 'tuner.py'

In [51]:
# Setup the Tuner component  /// https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/Tuner
tuner = tfx.components.Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=512),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )

In [53]:
# Run the component. This will take around 10 minutes to run.
# When done, it will summarize the results and show the 10 best trials.
context.run(tuner, enable_cache=False)

Trial 1 Complete [00h 00m 20s]
val_loss: 0.7415475249290466

Best val_loss So Far: 0.7415475249290466
Total elapsed time: 00h 00m 20s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
470               |315               |units_1
250               |400               |units_2
500               |325               |units_3
200               |100               |units_4
0.001             |0.0001            |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
512/512 [==============================] - 14s 26ms/step - loss: 0.7889 - categorical_accuracy: 0.5524 - val_loss: 0.6897 - val_categorical_accuracy: 0.4634
Epoch 2/2
 22/512 [>.............................] - ETA: 9s - loss: 0.6379 - categorical_accuracy: 0.5114

KeyboardInterrupt: 

## Trainer

In [47]:
# Declare trainer module file
_trainer_module_file = 'trainer.py'

In [15]:
# %%writefile {_trainer_module_file}
# Import necessary libraries
from tensorflow import keras
from kerastuner.engine import base_tuner
import os
from pathlib import Path
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Constants from housing_constants_module_file
import housing_constant
_transformed_name = housing_constant.transformed_name

# defining a structure(NamedTuple) that can hold both the tuner tool (base_tuner) and the training instructions(fit_kwargs) together
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])
# defining the label
LABEL_KEY = "ocean_proximity_xf"




# 1. Load compressed dataset
def _gzip_reader_fn(filenames):  # helper function to load data 
  '''Load compressed dataset

  Args:
    filenames - filenames of TFRecords to load

  Returns:
    TFRecordDataset loaded from the filenames
  '''
#Loading the dataset. Specify the compression type since it is saved as `.gz`
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')




# 2. Data loading for model training and validation is performed in batches, and the loading is handled by the input_fn()
def _input_fn(file_pattern,     # helper function to load data and preprocessing 
              tf_transform_output,
              num_epochs = None,
              batch_size=32) -> tf.data.Dataset:
  '''Create batches of features and labels from TF Records

  Args:
    file_pattern - List of files or patterns of file paths containing Example records.
    tf_transform_output -The transform output graph from TensorFlow Transform (TF Transform) that contains information about how features have been transformed or preprocessed.
    num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
    batch_size - An int representing the number of records to combine in a single batch.

  Returns:
    A dataset of dict elements, (or a tuple of dict elements and label). 
    Each dict maps feature keys to Tensor or SparseTensor objects.
  '''

  # Get feature specification based on transform output = This specification defines the structure and types of features after they have been transformed by TF Transform.
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  # Create batches of features and labels
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=LABEL_KEY)
  
  return dataset
# The input_fn returns a generator (a batched_features_dataset) that will supply
# data to the model one batch at a time.



# 3. Applying the preprocessing graph to model inputs
# This function bridges the gap between your trained model and its deployment for serving predictions. 
# It essentially defines how the model handles incoming requests containing new data points.

#Signatures specify how the model can be used for inference, including input and output formats.
def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    model.tft_layer = tf_transform_output.transform_features_layer() #Load the preprocessing graph

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(housing_constant.LABEL_KEY)
        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec  #Parse the raw tf.Example records from the request.
        )

        transformed_features = model.tft_layer(parsed_features) #Apply the preprocessing transformation to raw data

        outputs = model(transformed_features) #Perform prediction with preprocessed data.
        return {"outputs": outputs}

    return serve_tf_examples_fn




# 4. DEFINING MODEL ARCHITECTURE
def build_model(hp):
  """Builds a Keras model for housing price prediction with hyperparameter tuning.

  Args:
      hp: Hyperparameters object from KerasTuner.

  Returns:
      A compiled Keras model.
  """

  input_features = []
  for x in housing_constant.NUMERIC_FEATURE_KEYS:
    input_features.append(tf.keras.Input(shape=(1,), name=_transformed_name(x)))

  for x in housing_constant.BUCKET_FEATURE_KEYS:
    input_features.append(tf.keras.Input(shape=(1,), name=_transformed_name(x)))  # Bucket features have shape (1,) after transformation

  # Concatenate all input layers
  concat = tf.keras.layers.concatenate(input_features)

  # Hidden layers
  first_layer = tf.keras.layers.Dense(units=hp.get('units_1'), activation='relu')(concat)
  dropout_layer = tf.keras.layers.Dropout(0.2)(first_layer)
  second_layer = tf.keras.layers.Dense(units=hp.get('units_2'), activation='relu')(dropout_layer)
  third_layer = tf.keras.layers.Dense(units=hp.get('units_3'), activation='relu')(second_layer)

  # Output layer
  output_layer = tf.keras.layers.Dense(1, activation='softmax')(third_layer)

  # Create the model
  model = tf.keras.Model(inputs=input_features, outputs=output_layer)

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.get('learning_rate') 
                                                ) ,loss='mse',metrics=['accuracy'])
  # Print model summary for debugging purposes (optional)
  model.summary()

  return model




# 5. Training Orchestration
# run_fn is the main function that coordinates training. It calls functions in a specific order to:
# Load the transform graph (tf_transform_output)
# Prepare training and validation datasets using _input_fn with the transform graph
# Load hyperparameters from fn_args
# Build the model using build_model with loaded hyperparameters
# Train the model using model.fit with prepared datasets and callbacks
# Save the model with signatures using model.save


#The Trainer component will look for a run_fn() function in our module file and use
#the function as an entry point to execute the training process. 
#The module file needs The TFX Trainer Component to be accessible to the Trainer component

def run_fn(fn_args: FnArgs) -> None:
  """Defines and trains the model.
  Args:
    fn_args: Holds args as name/value pairs. fn_args: This is an object of type FnArgs that holds arguments as name/value pairs. 
    It contains various attributes such as paths to transform graphs, example datasets, hyperparameters, and model directories
    Refer here for the complete attributes: 
    https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
  """

#the run_fn function receives a set of arguments, including the transform
#graph, example datasets, and training parameters through the fn_args object
    
  # 5.1 Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  # Create batches of data good for 10 epochs
  train_set = _input_fn(fn_args.train_files, tf_transform_output, 10)
  val_set = _input_fn(fn_args.eval_files, tf_transform_output, 10)

  # 5.2 Load best hyperparameters
  hp = fn_args.hyperparameters.get('values')
    

  # 5.3 Build the model
  model = build_model(hp)

  #Callbacks
  log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir, update_freq='batch')
    
  # 5.4 Train the model
  model.fit(x=train_set,validation_data=val_set , callbacks=[tensorboard_callback])
#Signatures specify how the model can be used for inference, including input and output formats.
  signatures = {'serving_default': 
                _get_serve_tf_examples_fn(model,tf_transform_output).get_concrete_function(tf.TensorSpec(shape=[None],
                                                                                                         dtype=tf.string,name='examples'))}
  # 5.5 Save the model
  model.save(fn_args.serving_model_dir, save_format='tf',signatures=signatures) 
  # model_name = "my_housing_model"
  # model_version = "0002"
  # model_path = Path(model_name) / model_version   #another method to save the model
  # model.save(model_path, save_format="tf")

Overwriting trainer.py


In [48]:
# Setup the Trainer component
_trainer_module_file = 'trainer.py'
trainer = tfx.components.Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']))

In [50]:
# Run the component
context.run(trainer, enable_cache=False)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 longitude_xf (InputLayer)   [(None, 1)]                  0         []                            
                                                                                                  
 latitude_xf (InputLayer)    [(None, 1)]                  0         []                            
                                                                                                  
 total_rooms_xf (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 total_bedrooms_xf (InputLa  [(None, 1)]                  0         []                      

NameError: name '_get_tf_examples_serving_signature' is not defined

## LOADING THE  MODEL PUSHED BY PUSHER COMPONENT FOR SERVVING

In [15]:
model_dir  = "serving_model_dir/1716886967"

In [16]:
model_dir 

'serving_model_dir/1716886967'

In [17]:
#  model_artifact_dir = trainer.outputs['model'].get()[0].uri
# # log_dir = os.path.join(model_artifact_dir, 'logs')
# log_dir = '../pipeline_root_learning/tfx_for_tfrecordData/Trainer/model/220/logs'
# %reload_ext tensorboard
# %tensorboard --logdir {log_dir} 

## Using the SavedModel outside a pipeline

###### In a complete pipeline, if you're using TensorFlow Serving for inference, you typically don't need to load the model manually using tf.saved_model.load as shown above. TensorFlow Serving handles the loading and serving of the model automatically based on the configuration you provide.

In [18]:
import tensorflow as tf
# If we would like to inspect the exported SavedModel outside a TFX pipeline, we can
# load the model as a concrete function,5 which represents the graph of a single
# signature:
model = tf.saved_model.load(export_dir=model_dir)
predict_fn = model.signatures["serving_default"]
predict_fn 

<ConcreteFunction signature_wrapper(*, housing_median_age_xf, longitude_xf, median_house_value_xf, households_xf, latitude_xf, median_income_xf, population_xf, total_rooms_xf, total_bedrooms_xf) at 0x26C34D1C9D0>

In [19]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

# Prepare your input data (assuming you have a single data point for prediction)
input_data = {
    "longitude": 11111110,
    "latitude": 1111110,
    "housing_median_age": 111110,
    "total_rooms": 111110,
    "total_bedrooms":145011111,
    "population": 45455440,
    "households": 4554540,
    "median_income": 4545450.3252,
    "median_house_value":454545540
}

# Convert input data to tf.train.Example format
example = tf.train.Example(features=tf.train.Features(feature={
    "longitude": _float_feature(input_data["longitude"]),
    "latitude": _float_feature(input_data["latitude"]),
    "housing_median_age": _float_feature(input_data["housing_median_age"]),
    "total_rooms": _float_feature(input_data["total_rooms"]),
    "total_bedrooms": _float_feature(input_data["total_bedrooms"]),
    "population": _float_feature(input_data["population"]),
    "households": _float_feature(input_data["households"]),
    "median_income": _float_feature(input_data["median_income"]),
    "median_house_value": _float_feature(input_data["median_house_value"])
}))

# Serialize the example
serialized_example = example.SerializeToString()

In [20]:
serialized_example

b'\n\xed\x01\n\x1e\n\x12housing_median_age\x12\x08\x12\x06\n\x04\x00\x03\xd9G\n\x15\n\tlongitude\x12\x08\x12\x06\n\x04\xc6\x8a)K\n\x14\n\x08latitude\x12\x08\x12\x06\n\x040\xa2\x87I\n\x16\n\nhouseholds\x12\x08\x12\x06\n\x04X\xfe\x8aJ\n\x1e\n\x12median_house_value\x12\x08\x12\x06\n\x04\x84\xbe\xd8M\n\x16\n\npopulation\x12\x08\x12\x06\n\x04\x14f-L\n\x1a\n\x0etotal_bedrooms\x12\x08\x12\x06\n\x04\x1aK\nM\n\x19\n\rmedian_income\x12\x08\x12\x06\n\x04U\xb7\x8aJ\n\x17\n\x0btotal_rooms\x12\x08\x12\x06\n\x04\x00\x03\xd9G'

In [28]:
print(predict_fn(tf.constant([example])))

ValueError: Attempt to convert a value (features {
  feature {
    key: "households"
    value {
      float_list {
        value: 4554540.0
      }
    }
  }
  feature {
    key: "housing_median_age"
    value {
      float_list {
        value: 111110.0
      }
    }
  }
  feature {
    key: "latitude"
    value {
      float_list {
        value: 1111110.0
      }
    }
  }
  feature {
    key: "longitude"
    value {
      float_list {
        value: 11111110.0
      }
    }
  }
  feature {
    key: "median_house_value"
    value {
      float_list {
        value: 454545536.0
      }
    }
  }
  feature {
    key: "median_income"
    value {
      float_list {
        value: 4545450.5
      }
    }
  }
  feature {
    key: "population"
    value {
      float_list {
        value: 45455440.0
      }
    }
  }
  feature {
    key: "total_bedrooms"
    value {
      float_list {
        value: 145011104.0
      }
    }
  }
  feature {
    key: "total_rooms"
    value {
      float_l

## Inspecting Exported Modelstions.

`Inspect the signatures of exported models`

This is very useful primarily when you don’t export the model yourself, and you
want to learn about the inputs and outputs of the model graph.



`Test the exported models`


The CLI tools let you infer the model without deploying it with TensorFlow
Serving. This is extremely useful when you want to test your model input data.
We’ll cover both use cases in the following two sections.

In [23]:
#!pip install tensorflow-serving-api
path_to_savedmodel = model_dir
!saved_model_cli show --dir {path_to_savedmodel} 

The given SavedModel contains the following tag-sets:
'serve'


In [24]:
#Once you know the tag_set we want to inspect it...
!saved_model_cli show --dir {path_to_savedmodel} --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


`SavedModel MetaGraphDef`: This refers to the overall structure and information about the saved model. It's like a blueprint that describes what's inside the model file.

`SignatureDefs`: These are definitions that specify how the model can be used. They act as instructions on how to interact with the model and what kind of inputs and outputs it expects.

`SignatureDef keys`: These keys identify the specific types of functionalities provided by the model. In this case, you see two keys:

    "__saved_model_init_op": This signature typically defines an initialization operation that might be necessary to prepare the model before using it. It's like a warm-up step for the model.
    
    "serving_default": This signature likely defines the main functionality of the model used for making predictions. It's the "workhorse" part that takes input data, performs calculations, and generates output predictions.

In [25]:
# With the tag_set and signature_def information, you can now inspect the model’s
# inputs and outputs

!saved_model_cli show --dir {path_to_savedmodel} \
--tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['households_xf'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_households_xf:0
  inputs['housing_median_age_xf'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_housing_median_age_xf:0
  inputs['latitude_xf'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_latitude_xf:0
  inputs['longitude_xf'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_longitude_xf:0
  inputs['median_house_value_xf'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_median_house_value_xf:0
  inputs['median_income_xf'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_median_income_xf:0
  inputs['population_xf'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_population_xf:0
  inputs['total_bed

##### Understanding a SavedModel's Inputs and Outputs

The provided message describes the structure of a TensorFlow SavedModel, which is a format for packaging machine learning models for deployment. It details the inputs the model expects and the outputs it produces.

Breakdown:

SavedModel SignatureDef: This section refers to the definition of how the model can be used for prediction. It specifies the inputs, outputs, and potentially other details related to making predictions with the model.

`Inputs:`

Name: examples

Data Type: DT_STRING (text strings)

Shape: (-1,) (any number of strings)

Description: The model expects a tensor (a multi-dimensional array) of strings as input. This likely represents the data samples you want the model to make predictions on. Each string in the tensor could be a sentence, a record, or any other data point formatted as text.

`Outputs:`

Name: outputs

Data Type: DT_FLOAT (floating-point numbers)

Shape: (-1, 1) (any number of rows, but only one column)

Description: The model produces a tensor of floating-point numbers as output. It likely contains the model's predictions for each input sample. Since the shape has one column, the model is likely generating a single prediction value per input sample.

Method Name: tensorflow/serving/predict

`Description:` This indicates that the model is intended for prediction tasks using TensorFlow Serving, a framework for serving machine learning models.

In [23]:
# If we want to see all signatures regardless of the tag_set and signature_def, we
# can use the --all argument:
!saved_model_cli show --dir {path_to_savedmodel} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['examples'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_examples:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['outputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall_2:0
  Method name is: tensorflow/serving/predict
The MetaGraph with tag set ['serve'] contains the following ops: {'ParseExampleV2', 'StringJoin', 'StatefulPartitionedCall', 'Placeholder'

2024-05-21 19:22:58.587067: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Setting Up TensorFlow Serving

### TensorFlow Serving with Docker

In [ ]:
# !docker pull tensorflow/serving

In [6]:
# i have saved the model with two different approch in trainer script and utilizing the second approch's saved madel loaction for tensorflow serving
!docker run -p 8500:8500 \
-p 8501:8501 \
--mount type=bind,source="C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder\my_housing_model",target=/models/my_model \
-e MODEL_NAME=my_model -t tensorflow/serving 
#source path should be absolute means from root directory i.e
#path should not contain model version 

2024-05-24 13:37:31.977523: I tensorflow_serving/model_servers/server.cc:77] Building single TensorFlow model file config:  model_name: my_model model_base_path: /models/my_model
2024-05-24 13:37:31.981729: I tensorflow_serving/model_servers/server_core.cc:471] Adding/updating models.
2024-05-24 13:37:31.981787: I tensorflow_serving/model_servers/server_core.cc:600]  (Re-)adding model: my_model
2024-05-24 13:37:32.309964: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: my_model version: 2}
2024-05-24 13:37:32.310063: I tensorflow_serving/core/loader_harness.cc:68] Approving load for servable version {name: my_model version: 2}
2024-05-24 13:37:32.310111: I tensorflow_serving/core/loader_harness.cc:76] Loading servable version {name: my_model version: 2}
2024-05-24 13:37:32.354065: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /models/my_model/0002
2024-05-24 13:37:32.384545: I external/

In [7]:
!docker ps


CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [27]:
## we have successfully serve the model!!

#### Multiple Model Configuration

configure TensorFlow Serving to load multiple models at the same time.
To do that, we need to create a configuration file to specify the models:
see pdf...

In [21]:
# !docker run -p 8500:8500 \
# -p 8501:8501 \
# --mount type=bind,source="C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder\file_created_to_serve_multiple_model",target=/models/my_model \
# --mount type=bind,source="C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder\model_config",target=/models/model_config \
# -e MODEL_NAME=my_model -t tensorflow/serving  --model_config_file=/models/model_config/model.config 

## Making Predictions from the Model Server

#### Getting Model Predictions via REST

In [28]:
import requests
import numpy as np

def get_rest_request(instances, model_name="my_model"):
    url = f"http://localhost:8501/v1/models/{model_name}:predict"
    
    # Create the payload in the expected format
    payload = {
        "inputs": {
            "longitude_xf": np.array([instance['longitude'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "latitude_xf": np.array([instance['latitude'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "housing_median_age_xf": np.array([instance['housing_median_age'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "total_rooms_xf": np.array([instance['total_rooms'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "total_bedrooms_xf": np.array([instance['total_bedrooms'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "population_xf": np.array([instance['population'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "households_xf": np.array([instance['households'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "median_income_xf": np.array([instance['median_income'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "median_house_value_xf": np.array([instance['median_house_value'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist()
        }
    }
    
    response = requests.post(url=url, json=payload)
    return response

# Input data as a list of dictionaries
input_data = [
    {
        "longitude": 11111110,
        "latitude": 1111110,
        "housing_median_age": 111110,
        "total_rooms": 111110,
        "total_bedrooms": 145011111,
        "population": 45455440,
        "households": 4554540,
        "median_income": 4545450.3252,
        "median_house_value": 454545540
    },
    {
        "longitude": 22222220,
        "latitude": 2222220,
        "housing_median_age": 222220,
        "total_rooms": 222220,
        "total_bedrooms": 145022222,
        "population": 45455441,
        "households": 4554541,
        "median_income": 4545451.3252,
        "median_house_value": 454545541
    }
]

# Make the prediction request
rs_rest = get_rest_request(input_data)
response_json = rs_rest.json()
print(response_json)


{'outputs': [[1.0], [1.0], [1.0], [1.0]]}
2024-05-22 07:17:53.106164: I tensorflow_serving/model_servers/server.cc:77] Building single TensorFlow model file config:  model_name: my_model model_base_path: /models/my_model
2024-05-22 07:17:53.110319: I tensorflow_serving/model_servers/server_core.cc:471] Adding/updating models.
2024-05-22 07:17:53.110392: I tensorflow_serving/model_servers/server_core.cc:600]  (Re-)adding model: my_model
2024-05-22 07:17:53.446165: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: my_model version: 2}
2024-05-22 07:17:53.446297: I tensorflow_serving/core/loader_harness.cc:68] Approving load for servable version {name: my_model version: 2}
2024-05-22 07:17:53.446402: I tensorflow_serving/core/loader_harness.cc:76] Loading servable version {name: my_model version: 2}
2024-05-22 07:17:53.465892: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /models/my_model/00

`Explanation:`

Dictionary of Lists: The payload is constructed as a dictionary of lists. Each key in the dictionary corresponds to a tensor name expected by the model (e.g., "longitude_xf", "latitude_xf", etc.), and each value is a list of values for that tensor.

np.array and tolist(): The values for each key are converted to lists using numpy to ensure compatibility with the expected input format.

This approach aligns the input data format with the tensor names and shapes specified in the MetaGraphDef for your model. Try running this revised code to see if it resolves the error.

#### if we wish to make a request from postman API , we need to pass the input in differnt format specifically tensors

The reason for sending the input data in this specific format is to comply with the requirements of TensorFlow Serving for handling batch predictions and the expected input tensor structure. Here's a detailed explanation:

### TensorFlow Serving Input Format

TensorFlow Serving expects input data to be formatted as tensors. Each tensor can be thought of as a multi-dimensional array. The format we use here ensures that the data aligns with the expected input signature of the model.

### Breaking Down the Format

1. **Input Key Names**: The input key names (`longitude_xf`, `latitude_xf`, etc.) correspond to the tensor names defined in your model. These names must match exactly with the names expected by the model.

2. **Batch Dimension**: The outer brackets `[ ]` in each value represent the batch dimension. TensorFlow models typically expect input data to be batched, even if you are sending only one instance for prediction. Batching allows for processing multiple instances simultaneously, which is efficient for both training and inference.

3. **Inner Values**: The inner brackets `[[ ]]` around each value are necessary to ensure that each input feature is treated as a tensor with a shape of `(batch_size, 1)`. Here, `batch_size` is the number of instances you are predicting in one go. Each instance is a row in this tensor.

### Example Structure

To illustrate this with your example:

- For a single feature like `longitude`, the format `[[11111110.0], [22222220.0]]` means you have a batch size of 2, with each instance having a single value for `longitude`.
- The resulting tensor shape for each feature is `(batch_size, 1)`.

This approach ensures that the model receives the input in the correct shape, avoiding errors related to dimension mismatches.

### Why It Matters

TensorFlow Serving processes inputs in tensor format, and each tensor should conform to the model's expected shape. The batching mechanism (using outer brackets) allows TensorFlow Serving to efficiently process multiple instances in parallel.

### Summary

Here's your JSON payload again with this understanding:

```json
{
  "inputs": {
    "longitude_xf": [[11111110.0], [22222220.0]],
    "latitude_xf": [[1111110.0], [2222220.0]],
    "housing_median_age_xf": [[111110.0], [222220.0]],
    "total_rooms_xf": [[111110.0], [222220.0]],
    "total_bedrooms_xf": [[145011111.0], [145022222.0]],
    "population_xf": [[45455440.0], [45455441.0]],
    "households_xf": [[4554540.0], [4554541.0]],
    "median_income_xf": [[4545450.3252], [4545451.3252]],
    "median_house_value_xf": [[454545540.0], [454545541.0]]
  }
}
```

This format ensures compatibility with TensorFlow Serving, allowing for correct processing of your prediction request.

### Using TensorFlow Serving via gRPC

In [139]:
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import tensorflow as tf

# Establish the channel
channel = grpc.insecure_channel('localhost:8500')

# Create the stub
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

# Prepare the request
request = predict_pb2.PredictRequest()
request.model_spec.name = 'my_model'
# Add input data
request.inputs['longitude_xf'].CopyFrom(tf.make_tensor_proto([[11111110.0], [22222220.0]]))
request.inputs['latitude_xf'].CopyFrom(tf.make_tensor_proto([[1111110.0], [2222220.0]]))
request.inputs['housing_median_age_xf'].CopyFrom(tf.make_tensor_proto([[111110.0], [222220.0]]))
request.inputs['total_rooms_xf'].CopyFrom(tf.make_tensor_proto([[111110.0], [222220.0]]))
request.inputs['total_bedrooms_xf'].CopyFrom(tf.make_tensor_proto([[145011111.0], [145022222.0]]))
request.inputs['population_xf'].CopyFrom(tf.make_tensor_proto([[45455440.0], [45455441.0]]))
request.inputs['households_xf'].CopyFrom(tf.make_tensor_proto([[4554540.0], [4554541.0]]))
request.inputs['median_income_xf'].CopyFrom(tf.make_tensor_proto([[4545450.3252], [4545451.3252]]))
request.inputs['median_house_value_xf'].CopyFrom(tf.make_tensor_proto([[454545540.0], [454545541.0]]))

# Make the prediction
response = stub.Predict(request)

# Process the response
print(response)


outputs {
  key: "dense_15"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 2
      }
      dim {
        size: 1
      }
    }
    float_val: 1.0
    float_val: 1.0
  }
}
model_spec {
  name: "my_model"
  version {
    value: 2
  }
  signature_name: "serving_default"
}

2024-05-21 17:08:22.542107: I tensorflow_serving/model_servers/server.cc:77] Building single TensorFlow model file config:  model_name: my_model model_base_path: /models/my_model
2024-05-21 17:08:22.561382: I tensorflow_serving/model_servers/server_core.cc:471] Adding/updating models.
2024-05-21 17:08:22.561507: I tensorflow_serving/model_servers/server_core.cc:600]  (Re-)adding model: my_model
2024-05-21 17:08:22.852923: I tensorflow_serving/core/basic_manager.cc:740] Successfully reserved resources to load servable {name: my_model version: 2}
2024-05-21 17:08:22.853005: I tensorflow_serving/core/loader_harness.cc:68] Approving load for servable version {name: my_model version: 2}
2024-05-21

### Batching Inference Requests

In [35]:
!docker run -p 8500:8500 \
-p 8501:8501 \
--mount type=bind,source="C:\Users\ASUS\building-machine-learning-pipelines\Untitled Folder\my_housing_model",target=/models/my_model \
--mount type=bind,source="C:\Users\ASUS\building-machine-learning-pipelines/Untitled Folder/model_config/",target=/server_config \
-e MODEL_NAME=my_model -t tensorflow/serving \
--enable_batching=true --batching_parameters_file=/server_config/batching_parameters.txt --log-verbose


^C


In [36]:
## To verify that batching is working in TensorFlow Serving, 
##we need to send multiple requests and check that they are being batched together as expected
import requests
import numpy as np
import threading
import time

def get_rest_request(instances, model_name="my_model"):
    url = f"http://localhost:8501/v1/models/{model_name}:predict"
    
    # Create the payload in the expected format
    payload = {
        "inputs": {
            "longitude_xf": np.array([instance['longitude'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "latitude_xf": np.array([instance['latitude'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "housing_median_age_xf": np.array([instance['housing_median_age'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "total_rooms_xf": np.array([instance['total_rooms'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "total_bedrooms_xf": np.array([instance['total_bedrooms'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "population_xf": np.array([instance['population'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "households_xf": np.array([instance['households'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "median_income_xf": np.array([instance['median_income'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "median_house_value_xf": np.array([instance['median_house_value'] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist()
        }
    }
    
    response = requests.post(url=url, json=payload)
    return response

# Input data as a list of dictionaries
input_data_1 = {
    "longitude": 11111110,
    "latitude": 1111110,
    "housing_median_age": 111110,
    "total_rooms": 111110,
    "total_bedrooms": 145011111,
    "population": 45455440,
    "households": 4554540,
    "median_income": 4545450.3252,
    "median_house_value": 454545540
}

input_data_2 = {
    "longitude": 22222220,
    "latitude": 2222220,
    "housing_median_age": 222220,
    "total_rooms": 222220,
    "total_bedrooms": 145022222,
    "population": 45455441,
    "households": 4554541,
    "median_income": 4545451.3252,
    "median_house_value": 454545541
}

def send_request():
    rs_rest = get_rest_request([input_data_1, input_data_2])
    response_json = rs_rest.json()
    print(response_json)

# Create multiple threads to send requests quickly
threads = []
for _ in range(5):
    t = threading.Thread(target=send_request)
    t.start()
    threads.append(t)
    time.sleep(0.01)  # slight delay to simulate rapid requests

# Wait for all threads to complete
for t in threads:
    t.join()


{'outputs': [[1.0], [1.0]]}
{'outputs': [[1.0], [1.0]]}
{'outputs': [[1.0], [1.0]]}
{'outputs': [[1.0], [1.0]]}
{'outputs': [[1.0], [1.0]]}


#### If we decided to pass only the input values we can do in this manner

In [39]:
import requests
import numpy as np

def get_rest_request(instances, model_name="my_model"):
    url = f"http://localhost:8501/v1/models/{model_name}:predict"
    
    # Create the payload in the expected format
    payload = {
        "inputs": {
            "longitude_xf": np.array([instance[0] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "latitude_xf": np.array([instance[1] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "housing_median_age_xf": np.array([instance[2] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "total_rooms_xf": np.array([instance[3] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "total_bedrooms_xf": np.array([instance[4] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "population_xf": np.array([instance[5] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "households_xf": np.array([instance[6] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "median_income_xf": np.array([instance[7] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist(),
            "median_house_value_xf": np.array([instance[8] for instance in instances], dtype=np.float32).reshape(-1, 1).tolist()
        }
    }
    
    response = requests.post(url=url, json=payload)
    return response

# Input data as a list of lists (values only)
input_data = [
    [11111110, 1111110, 111110, 111110, 145011111, 45455440, 4554540, 4545450.3252, 454545540],
    [22222220, 2222220, 222220, 222220, 145022222, 45455441, 4554541, 4545451.3252, 454545541]
]

# Make the prediction request
rs_rest = get_rest_request(input_data)
response_json = rs_rest.json()
print(response_json)


{'outputs': [[1.0], [1.0]]}
unknown argument: --log-verbose
usage: tensorflow_model_server
Flags:
	--port=8500                      	int32	TCP port to listen on for gRPC/HTTP API. Disabled if port set to zero.
	--grpc_socket_path=""            	string	If non-empty, listen to a UNIX socket for gRPC API on the given path. Can be either relative or absolute path.
	--rest_api_port=0                	int32	Port to listen on for HTTP/REST API. If set to zero HTTP/REST API will not be exported. This port must be different than the one specified in --port.
	--rest_api_num_threads=64        	int32	Number of threads for HTTP/REST API processing. If not set, will be auto set based on number of CPUs.
	--rest_api_timeout_in_ms=30000   	int32	Timeout for HTTP/REST API calls.
	--rest_api_enable_cors_support=false	bool	Enable CORS headers in response
	--enable_batching=false          	bool	enable batching
	--allow_version_labels_for_unavailable_models=false	bool	If true, allows assigning unused version